# Introduction

Below is a 'clean' version of the Jupyter Notebook used in Basic Skills Air Quality. You can use it if you want to work with ammonia deposition on a land or water surface.

In your application, consider the following points:
1. Summarise the methods in your PBL report
2. Document all choices you make with respect to parameter settings and processes included (or omitted)
3. Remember that a model is 'just' a set of numerical rules. You have to interpret the results considering meaning, reliability, accuracy
4. Consider which parameters or processes may be most sensitive and/or unreliable and test how sensitive the model is to those. Document the sensitivity as a confidence interval: in which interval are you confident that the model results will be?
5. **NEVER** weaken your report by writing things like: 'Due to uncertainty in x, the model results are uncertain.' In this way you entirely undermine your model results. **Instead**, document how reliable the model is in terms of a confidence interval (see point 4).

## Things to check:
- did you use the correct parameters for your land use type (parameterisations for $r_a$ and $r_c$?
- how about implementing stability corrections for $r_a$ (see MAQ10306 Introduction Atmosphere)?
- how about implementing stress functions F2, F3 and F4 for $r_c$?

### 1. Load Python modules

In [ ]:
# Load necessary python packages.
import sys
!{sys.executable} -m pip install cufflinks > /dev/null; # Remove > /dev/null in case of errors.

from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
import plotly.express as px

# (Ignore the pip version warning)

### 2. Load input data

In [ ]:
# Load the values of nitrogen NH3 concentration for three different stations (Wekerom, Vredepeel and Zegveld). 
# Data are loaded as hourly values in ug/m3.
# N.B.: Do not try to hard to understand all of the code. We will mention which parts are important to understand.

df_Wekerom = pd.read_csv('../Data_IntegrationCourseSWA/Data_AQ_Wekerom_2013-2019.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Wekerom.columns = ['NH3_W']

df_Vredepeel = pd.read_csv('../Data_IntegrationCourseSWA/Data_AQ_Vredepeel_2013-2019.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Vredepeel.columns = ['NH3_V']

df_Zegveld = pd.read_csv('../Data_IntegrationCourseSWA/Data_AQ_Zegveld_2013-2019.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Zegveld.columns = ['NH3_Z']

# Save the data from Wekerom, Vredepeel and Zegveld in the same data frame.
df_AQ = pd.concat([df_Wekerom, df_Vredepeel, df_Zegveld], axis=1, sort=False)

In [ ]:
# Load the meteorological data. 
# Load the velocity (u; m/s), global radiation (Rg; W/m^2), rain (Rain; mm), rain in the last three hours 
# (Rain_last3h; mm) and leaf area index (LAI; m^2/m^2).
df_meteo = pd.read_csv('../Data_IntegrationCourseSWA/Data_Meteo_Veenkampen_2013-2019.csv', sep=';', 
    index_col='date-time', usecols=['date-time','Rg','Ta','RH','u','P','smc_065','smc_125','smc_250','smc_500'],
                           parse_dates=['date-time'])
df_meteo.columns = ['Rg','Ta','RH','u','P','smc_065','smc_125','smc_250','smc_500']

# Units:
# Rg:   W/m2   Global radiation
# Ta:   oC     Air temperature at 2m
# RH:   %      Relative humidity
# u:    m/s    Wind speed at 10 m
# P:    mm/hr  Precipitation
# smc_* m3 water/m3 soil Soil moisture content at depth in mm

### 3. Physical constants and model parameters

In [ ]:
### Constants and Parameter Settings
#--- Collect all settings here to keep overview

### Physical constants

Rd         = 287.    # J/kg K Gas constant for dry air
Rv         = 462.    # J/kg K Gas constant for water vapour
p0         = 101.3   # kPa     Mean Air pressure at sea level

#--- Model parameters -------------------------------------------------------------------------------
# Set the constants needed for calculation of resistances for your land use type 
# in Kumar et al. (2001): https://link.springer.com/article/10.1007%2Fs10546-010-9559-z, Table 1; 
#----------------------------------------------------------------------------------------------------

#--- Parameters for r_a
k          = 0.4     # - Von Karman constant
z          = 10.     # m Reference height
zd         = 0.0     # m Displacement height
z0         = 0.0001  # m Roughness length Water

#--- Parameters for r_c
rcmin      = 40.0    # s/m    Minimum canopy resistance, when the stomates are fully open.
Rgl        = 100.0   # W/m2   Sensitivity to global radiation
LAI        = 2.0     # m2/m2  Leaf Area Index (in case of constant LAI, see 4.3.1)
rcmax      = 10000.  # s/m    Maximum canopy resistance, when the stomates are closed.
hs         = 47.35   # -      Sensitivity of rc to water vapor deficit
Tref       = 25.   # oC    Optimal temperature for photosynthesis
theta_wilt = 0.047 # m3/m3 Soil moisture content at wilting point
theta_ref  = 0.800 # m3/m3 Soil moisture content at field capacity
d1         = 0.095 # m     Thickness of soil layer 1
d2         = 0.093 # m     Thickness of soil layer 2
d3         = 0.188 # m     Thickness of soil layer 3
d4         = 0.200 # m     Thickness of soil layer 4
dtot       = 0.576 # m     total thickness of soil layer


#--- Calculate saturation vapor pressure and saturation water vapour mixing ratio
es0        = 0.6107   # kPa 
a          = 7.5      # -   a  parameter in Clausius-Clapeyron equation
b          = 237.3    # oC  b  parameter in Clausius-Clapeyron equation
es         = es0*10**(a*(df_meteo['Ta'])/(b+df_meteo['Ta']))   # kPa   Saturation vapor pressure
qs         = Rd/Rv * es / p0                                   # kg/kg Saturation vapor mixing ratio

#--- Calculate actual mixing ratio
qa         = df_meteo['RH']*qs / 100  # kg/kg actual water vapour mixing ratio


# 4. NH3 deposition

### 4.1 $r_a$: Aerodynamical resistance

In [ ]:
# Calculate the aerodynamical resistance. 
# 2. Calculate the aerodynamical resistance based on the formula given in section 3.1.
ra = (np.log((z-zd)/z0))**2/(k**2*df_meteo['u'])

### 4.2 $r_b$: Boundary layer resistance

In [ ]:
rb = 5.

### 4.3 $r_c$: Canopy resistance
<font color="blue">
Below you will find the formulation of the canopy resistance $r_c$ as in Basic Skills Air Quality. You may remember that we did a rather simplistic approach. Therefore we suggest a few extensions on how you can make the model more realistic. These are in <font color="blue">blue</font>.

### 4.3.1 Extension 1: Variable LAI
In the Basic Skills Air Quality, we used a constant LAI. In reality however, the LAI of most vegetation types changes (except evergreen needleleaf trees) with the seasons, and this has a large impact on the $r_c$. Here we suggest a way to prescribe a variable LAI.

In the summer season, the $LAI$ and NH$_3$ concentrations are larger than in the winter period. So there is a correlation between $r_c$ = f($LAI$) and $F_{NH_3}$. Omitting this could introduce a considerable error in the ammonia deposition flux. To account for variable $LAI$, you may implement a seasonal cycle in the $LAI$, using a cosine function as:

\begin{equation}
LAI_t = A_{LAI}+A_{LAI}\cos\Bigg(\frac{2\pi(t-t_{LAI,max})}{365}\Bigg)
\end{equation}

where $A_{LAI}$ (m$^2$m$^{-2}$)is the amplitude, $t$ (day) refers to the day of year and $t_{LAI,max}$ is the time (a day in a year) when the $LAI$ is the largest. Since the cosine part of the function varies between – $A_{LAI}$ and + $A_{LAI}$, we add one time $A_{LAI}$ to force the function to be positive. The $LAI$ thus varies between 0 and 2$A_{LAI}$. The 365 represents an annual cycle. 

Of course you can change the formulation any way you like (e.g. modify the amplitude or mean value or even introduce a sawtooth shape to simulating periodic mowing).

If introducing a variable LAI is important for you, please 
- change 'if False:' to 'if True:'
- review the formulation below
- inspect the figure to see if it the LAI is like you want it.
</font>

In [ ]:
### 4.3.1 Extension 1: Variable LAI
# Determine the amplitude of $LAI$ for the cosine function for your chosen vegetation type.
if False:
    A_lai     = 2.   # m2/m2 Amplitude of seasonal LAI
    t_lai_max = 181. # doy   The day of year with the largest LAI
  
    # Calculate the periodic (cosine) change in the $LAI$.
    df_meteo['LAI'] = A_lai + A_lai*np.cos(2.*np.pi*(df_meteo.index.dayofyear - t_lai_max) / 365.)
          
else:  # This is the default option with constant LAI as you initialised it in cel In [4]
    df_meteo['LAI'] = np.ones(len(df_meteo)) * LAI 
    
# Calculate the inter-annual seasonal variability of LAI
# Inter-annual seasonal variability can be calculated as a mean of all e.g., Januaries in our time series,
# or as a mean of all e.g., first weeks of the year.
LAI_seasonal = df_meteo['LAI'].groupby( 2*((df_meteo.index.week-1)//2 + 1)).mean()
  
# Plot the result (mean seasonal variability)
# Plot Time [weeks] on x-axis and LAI [m^2/m^2] on y-axis.
fig0 = LAI_seasonal.iplot(asFigure=True, xTitle="Time [weeks]", yTitle="LAI [m2/m2]", width=2)
fig0.show()


<font color="blue">


### 4.3.2 Extension 2: Stress functions for temperature, humidity and soil moisture.
Most vegetation types change the opening of their stomates to regulate how much CO$_2$ they take from the atmosphere and how much water vapor they lose. When the stomates are open, other species, such as NH$_3$ and O$_3$, can also be exchanged. When referring to the resistance at the leaf level, we use the term stomatal resistance. For tall vegetation structures, the stomatal resistance varies between the top and lower vegetation levels. Thus, when referring to the resistance at the vegetation level we use the term canopy resistance $r_c$ (s m$^{-1}$). The canopy resistance accounts for those differences (i.e., the differences between the top and lower vegetation levels).

There are two main approaches to model the canopy resistance. The first is the Jarvis approach. [Paul Jarvis](https://en.wikipedia.org/wiki/Paul_Gordon_Jarvis) (1935-2013) studied how canopy resistance responds to atmospheric conditions (e.g., vapor pressure deficit, radiation and temperature) and soil conditions (e.g., soil moisture and nutrient availability). He came up with a model formulation which assumes that each vegetation type has an intrinsic minimum resistance when the stomates are fully opened in optimal conditions. This minimum resistance is increased when the vegetation experiences stress, e.g., when the air is dry and the vegetation needs to reduce water loss. [Jarvis (1976)’s](https://royalsocietypublishing.org/doi/10.1098/rstb.1976.0035) formulation assumes that the vegetation response to each stress factor can be described by an independent multiplicative term. This makes the Jarvis approach simple to use and it also facilitates comparison between vegetation types.

<font color="blue">The second is the A-g$_s$ approach, where A stands for CO$_2$ assimilation and $g_s$ for canopy conductance, i.e., the inverse of canopy resistance (see e.g., [Jacobs and de Bruin, 1997](https://journals.ametsoc.org/doi/full/10.1175/1520-0450%281997%29036%3C1663%3APRTAEA%3E2.0.CO%3B2), [Ronda et al., 2001](https://journals.ametsoc.org/doi/full/10.1175/1520-0450%282001%29040%3C1431%3AROTCCI%3E2.0.CO%3B2)). This approach is based on newer scientific evidence that vegetation optimizes the stomatal resistance for the specific carbon uptake for minimal water loss. Some vegetation types respond rather different to environmental stress factors than others. For example, some species use water rather aggressively to grow faster, but at the risk of running out of water. Other species use water rather conservatively to prevent running out of water, but at the cost of growing slower (e.g., [van der Molen et al., 2010](https://www.sciencedirect.com/science/article/pii/S0168192311000517?via%3Dihub)). Taking into account the water cost of carbon uptake requires continuous integration between water availability, water loss and carbon uptake capacity of the vegetation. </font>

The Jarvis approach describes the canopy resistance as a minimum canopy resistance $r_{c,min}$ (s m$^{-1}$), which applies to optimal conditions. In stressed conditions, the actual canopy resistance $r_c$ (s m$^{-1}$) is larger than the minimum, depending on the stress terms $F_1$, $F_2$, ..., $F_n$:

\begin{equation*}
r_c = \frac{r_{c,min}}{LAI F_1F_2...F_n}.
\end{equation*}

The $LAI$ (m$^2$ m$^{-2}$) in the denominator is leaf-area index which implies that at each vegetation level $i$ with $LAI_i+1$, $r_c$ decreases linearly, which is arguably a simplistic approach, because lower levels may be shaded and cooler than higher layers. The stress functions $F_1$, ..., $F_n$ quantify the level of stress due to environmental processes. The stress functions are given as:

\begin{equation*}
F_1 = \frac{\frac{r_{c,min}}{r_{c,max}}+f}{1+f}, \\
F_2 = \frac{1}{1+h_s[q_s(T_a)-q_a]}, \\
F_3 = 1-0.0016(T_{ref}-T_a)^2, \\
F_4 = \sum_{i=1}^{n_{root}}\frac{(\theta_i-\theta_{wilt})d_i}{(\theta_{ref}-\theta_{wilt})d_{tot}}.
\end{equation*}


Here $r_{c,min}$ and $r_{c,max}$ (s m$^{-1}$) are minimum and maximum canopy resistances, respectively. We will initially work only with $F_1$ for simplicity, while neglecting $F_2$ to $F_4$. In the equation for $F_1$, $f=0.55\frac{R_g}{R_{gl}}\frac{2}{LAI}$, where $R_g$ (W m$^{-2}$) is global radiation, $R_{gl}$ (W m$^{-2}$) is the minimum solar radiation necessary for photosynthesis (transpiration) to occur. <font color="blue">Just to be complete, we give the descriptions of $F_2$ to $F_4$ too. In the equation for $F_2$, $h_s$ (-) is a parameter associated with the water vapour deficit, $T_a$ (K) is air temperature at reference level, $q_s$ and $q_a$ (g kg$^{-1}$ or kg kg$^{-1}$) are saturation and actual water vapor mixing ratios. They can be calculated as:

\begin{equation*}
q_s = \frac{R_d}{R_v}\frac{0.6107\cdot10^{\frac{7.5T_a}{237.3+T_a}}}{p}, \\
q_a = RH\cdot q_s,
\end{equation*}

where $R_d = 287$ J kg$^{-1}$ K$^{-1}$ and $R_v = 462$ J kg$^{-1}$ K$^{-1}$ are the gass constants for dry air and water vapour, respectively, $p \approx 1.1$ kPa is the air pressure at the sea level, and $RH$ (-) is the relative humidity. These parameters should be chosen in such a way that $F_2$ is a number between 1 and ~10. In the equation for $F_3$, $T_{ref}$ (K) is the optimal air temperature for photosynthesis, while $T_a$ (K) is the air temperature. Finally, in the equation for $F_4$, $\theta$ (m$^3$ m$^{-3}$) is volumetric water content, while and $d_i$ and $d_{tot}$ (m) are the depth of soil layer i and the total root depth.</font> This formulation and a more detailed description can be found in [Kumar et al. (2001)](https://link.springer.com/article/10.1007%2Fs10546-010-9559-z). Now, let's use this approach to simulate the canopy resistance $r_c$.
</font>

In [ ]:
# Calculate f and the stress function for solar radiation F1.
f_stress     = 0.55*(df_meteo['Rg']/Rgl)*(2./df_meteo['LAI'])
F1           = ((rcmin/rcmax)+f_stress)/(1+f_stress)
F1.name      = 'F1'

if False:
    # Calculate the stress function for temperature F2.
    F2       = 1./(1.+hs*(qs-qa))
    F2.name  = 'F2'

    # Calculate the stress function for soil moisture F3. 
    F3       = 1.-0.00166*(Tref-df_meteo['Ta'])**2.
    F3.name  = 'F3'

    # Calculate F4. First define some parameters. Parameters d1, ..., d4 and dtot are defined for you.
    denom    = (theta_ref - theta_wilt)*dtot
    F4_1     = ((df_meteo['smc_065']-theta_wilt)) / ((theta_ref - theta_wilt))
    F4_2     = ((df_meteo['smc_125']-theta_wilt)) / ((theta_ref - theta_wilt))
    F4_3     = ((df_meteo['smc_250']-theta_wilt)) / ((theta_ref - theta_wilt))
    F4_4     = ((df_meteo['smc_500']-theta_wilt)) / ((theta_ref - theta_wilt))
    F4       = (F4_1 * d1 + F4_2 * d2 + F4_3 * d3 + F4_4 * d4) / dtot
    F4.name  = 'F4'
else:
    F2       = 1.
    F3       = 1.
    F4       = 1.

# Make sure that F4 is in the range 0-1. This is a bit tricky, because:
# - you use smc measurements collected at the Veenkampen, where smc can be pretty high
# - if you set theta_wilt (wilting point ) to a value higher than the minimum smc in the measurements, F4 can become smaller than 0.
# - if you set theta_ref  (field capacity) to a value lower  than the maximum smc in the measurements, F4 can become larger  than 1.
# - so be careful when adapting theta_ref and theta_wilt. The theta_wilt and theta_ref can not be uncoupled from the observed smc.
print(F4)


In [ ]:
# Write the values of your chosen parameters using the land types given in Table 1 of Kumar et al, 2001.
# Numbers should represent the values for the minimum canopy resistance (rcmin; s/m), minimum global radiation for
# photosynthesis (Rgl; W/m^2), the leaf area index (LAI, m^2/m^2), and maximum canopy resistance (rcmax; s/m). 
# Please also take a look at the Python code, it may be useful to recognise what is being done here.

# Calculate the canopy resistance in s/m based on the formula given in the markdown-box above.
rc          = rcmin/(df_meteo['LAI'] *F1*F2*F3*F4)  # s/m Hourly canopy resistance, computed from the parameters supplied above.
rc.name     = 'rc'



<font color="blue">
    
### 4.3.3 Wet canopy resistance
After precipitation, the canopy is wet. Ammonia will deposit in the water on the leafs (and evaporation will be from the leaf surface instead of from the stomates). As a consequence, the stomatal resistance is bypassed ($r_c$ = 0 s m$^{-1}$). You may test how important this is by implementing this process.

In practise it is difficult to tell how long the canopy will remain wet. This depends on the canopy, the LAI, the temperature and the global radiation. But let's keep things simple for now, we assume the canopy remains wet until 3 hours after the last precipitation.
</font>

In [ ]:
rc_wet      = rc.copy(deep=True)                    # Initialise rc_wet as rc: rc_wet is the same as the canopy resistance rc
rc_wet.name = 'rc_wet'

# hours when it rains
I           = np.where(df_meteo['P'] > 0.0)[0]      # Find the index I of the   hours when P > 0.0
rc_wet[I]   = 0.0                                   # Set rc_wet = 0 for those hours when P > 0.0

# first hour after it rains
I           = I + 1                                  # Increase the index I with 1: select the first hour after precipitation occurred
I           = I[I < len(rc_wet)]                     # Make sure the index I cannot be longer than the length of rc_wet itself
rc_wet[I]   = 0.0                                    # Set rc_wet = 0 for those hours when P > 0.0

# second hour after it rains
I           = I + 1                                  # Increase the index I with 1: select the second hour after precipitation occurred
I           = I[I < len(rc_wet)]                     # Make sure the index I cannot be longer than the length of rc_wet itself
rc_wet[I]   = 0.0                                    # Set rc_wet = 0 for those hours when P > 0.0

# third hour after it rains
I           = I + 1                                  # Increase the index I with 1: select the third hour after precipitation occurred
I           = I[I < len(rc_wet)]                     # Make sure the index I cannot be longer than the length of rc_wet itself
rc_wet[I]   = 0.0                                    # Set rc_wet = 0 for those hours when P > 0.0

# Check out if it works
output = pd.concat([df_meteo['P'],rc,rc_wet],axis=1,sort=False)
I           = np.where(df_meteo['P'] > 0.0)[0]       # Find the instances when it rains   
i           = I[10]                                  # Find the 10th instance (just a choice) in the time series when it rains
print(output.iloc[i-10:i+10])

### 4.4 $r_t$: Total resistance

N.B.: Water surfaces do not have stomates and their $r_c$ = 0 s/m. Make sure you edit the following lines accordingly by setting rc = 0.0 or $r_t$ = $r_a$ + $r_b$

Additionally, you may need to replace $r_c$ with $r_{c,wet}$, if you want to account for zero stomatal conductance until a few hours after it rains.

In [ ]:
#--- Calculate the total conductance from ra, rb and rc or rc_wet (replace the symbol yourself)
rt = ra + rb + rc

### 4.5 $F_{NH_3}$: Ammonia deposition flux

In [ ]:
# Calculate the dry deposition rate (F_NH3; ug/m^2/s) using a gradient-resistance model for all three stations,
# i.e., for Wekerom, Vredepeel and Zegveld.
F_NH3_W = df_AQ['NH3_W']/rt
F_NH3_W.columns = ['F_NH3_W']

F_NH3_V = df_AQ['NH3_V']/rt
F_NH3_V.columns = ['F_NH3_V']

F_NH3_Z = df_AQ['NH3_Z']/rt
F_NH3_Z.columns = ['F_NH3_Z']


### 4.6 Some output options for you to tailor (figures, tables, datafiles)

In [ ]:
## Check content of a data frame
print(df_meteo.keys())
print(df_AQ   .keys())

### 4.6.1 Figure with mean seasonal cycle

In [ ]:
## Figure with mean seasonal cycle

# Calculate the inter-annual seasonal variability for all three stations. 
# Inter-annual seasonal variability can be calculated as a mean of all e.g., Januaries in our time series,
# or as a mean of all e.g., first weeks of the year.
df_mean_seasonal = df_AQ.groupby( 2*((df_AQ.index.week-1)//2 + 1)).mean()

# Plot the result (multi-year seasonal variability) for all three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [weeks] on x-axis and Concentration [um/m^3] on y-axis.
fig1 = df_mean_seasonal.iplot(asFigure=True, xTitle="Time [weeks]", yTitle="Concentration [ug/m3]", width=2)
fig1.show()

### 4.6.2 Figure with mean diurnal cycle

In [ ]:
## Mean diurnal cycle

# Calculate the multi-year daily variability in NH3 concentration for June and December for all three stations.
# Multi-year daily variability is calculated as a mean of all days in all 
# e.g., Junes and Decembers of all years in our time series.
# First, we extract June and December from our time series.
df_AQ_jun = df_AQ.loc[(df_AQ.index.month==6 )]
df_AQ_dec = df_AQ.loc[(df_AQ.index.month==12)]

# Next, we calculate the daily variability.
df_mean_jun_daily = df_AQ_jun.groupby([df_AQ_jun.index.hour]).mean()
df_mean_dec_daily = df_AQ_dec.groupby([df_AQ_dec.index.hour]).mean()

# Finally, we set up the new data (i.e., the daily variabilities) in a new matrix.
df_mean_jun_dec = pd.concat([df_mean_jun_daily, df_mean_dec_daily], axis=1, sort=False)
df_mean_jun_dec.columns = ['NH3_W_J', 'NH3_V_J', 'NH3_Z_J', 'NH3_W_D', 'NH3_V_D', 'NH3_Z_D']

# Plot the result (multi-year daily variability for June and December) for all three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Results for June are given with solid lines and results for December with dotted lines. 
# Plot Time [hours] on x-axis and Concentration [um/m^3] on y-axis. 
fig2 = df_mean_jun_dec.iplot(asFigure=True, xTitle="Time [hours]", yTitle="Concentration [um/m3]", 
    colors=['orange', 'blue', 'green', 'orange', 'blue', 'green'], 
    dash=['solid', 'solid', 'solid', 'dot', 'dot', 'dot'], width=2)
fig2.show()

### 4.6.3 Table with annual means

In [ ]:
## Table with annual means

# Fill in the variables you want to output.
df_output = pd.concat([F_NH3_W, F_NH3_V, F_NH3_Z],axis=1, sort=False)
df_output.columns = ['F_NH3_W','F_NH3_V', 'F_NH3_Z']
output = df_output[['F_NH3_W','F_NH3_V', 'F_NH3_Z']].groupby([df_output[['F_NH3_W']].index.year]).mean()
print(output)

### 4.6.4 Save output as a csv file

In [ ]:
## Save flux data to csv file

# I recommend to try and make the figure in this Notebook
# - if you change anything in the notebook, you only need to run the cell to update the figure
# - in Excel you need to reimport the data and make the figure all over again.
# - anyway, you may output the data in the following way:
df_output.to_csv('../Data_IntegrationCourseSWA/FluxNH3.csv', sep=';')

print('You can find the file in the filemanager (previous window) in the folder \"Data_IntegrationCourseSWA\".')

## 5.0 Close the notebook

Now, to limit the data storage please do the following:
- go to *KERNEL* --> *RESTART and CLEAR OUTPUT*. This removes the interactive graphs from your notebook, but leaves your answers intact.
- go to *FILE* --> *SAVE and CHECKPOINT*. This saves your answers in the notebook
- go to *FILE* --> *CLOSE and HALT*, to shutdown the notebook. You can always come back later and restart.